# Train and Test the Training Data!
As we discovered in the exploration phase, the kids with an sii score all have taken a questionaire and have had their score taken from that, which leaves the rest of the training data with the kids that don't have sii as they didn't take questionaire or rather their parents did not.<br>
As such we are left with a real training set of 2736 which is not huge, so we'll use cross-validation. <br>
Instinct is to use a nearest neighbours approach, so let's see how that goes.

In [2]:
#set up
import polars as pl
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


pl.Config.set_tbl_rows(50);
pl.Config.set_tbl_width_chars(300);
pl.Config.set_tbl_cols(90);

In [7]:
data = pl.read_csv('data/train.csv')
#seperate out into training and testing
test = data.filter(pl.col('sii').is_null()).fill_null(-1).fill_null("None")
#Remove the questionaire columns form training
dropCol = [col for col in data.columns if "PCIAT" in col or "Season" in col]
#dropSeasonCol = [col for col in data.columns if "Season" in col]
train = data.filter(pl.col('sii').is_not_null()).drop(dropCol).fill_null(-1).fill_null("None")

## KNN: Optimal value of K

In [16]:
# Split the data into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(train.drop('id','sii'), train.select('sii').to_numpy().flatten(), test_size=0.2, random_state=42)

# Define a range of k values to test
kValues = list(range(3,30))

#Dict to record testing in
kValueDict = {}

# Train and visualize the models with varying k values
for k in kValues:
    # Create KNN model
    knn = KNeighborsClassifier(n_neighbors=k)

    # Train the model
    knn.fit(Xtrain, ytrain)

    # Make predictions
    ypred = knn.predict(Xtest)

    # Calculate accuracy
    accuracy = accuracy_score(ytest, ypred)

    kValueDict[k] = accuracy



In [17]:
kValueDict

{3: 0.551094890510949,
 4: 0.5748175182481752,
 5: 0.5875912408759124,
 6: 0.5948905109489051,
 7: 0.5748175182481752,
 8: 0.5784671532846716,
 9: 0.5748175182481752,
 10: 0.5711678832116789,
 11: 0.5693430656934306,
 12: 0.5802919708029197,
 13: 0.5656934306569343,
 14: 0.5748175182481752,
 15: 0.5912408759124088,
 16: 0.593065693430657,
 17: 0.5912408759124088,
 18: 0.5967153284671532,
 19: 0.6040145985401459,
 20: 0.5894160583941606,
 21: 0.5857664233576643,
 22: 0.5784671532846716,
 23: 0.5857664233576643,
 24: 0.5875912408759124,
 25: 0.583941605839416,
 26: 0.583941605839416,
 27: 0.5894160583941606,
 28: 0.5802919708029197,
 29: 0.593065693430657}